# Step 1: Import Libraries and Load the CLIP Model

In [9]:
import torch
import os
import clip
from PIL import Image, UnidentifiedImageError
import requests
from io import BytesIO
import mysql.connector
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv